In [ ]:
!mkdir /root/.ssh
!mv /content/id_rsa /root/.ssh/id_rsa
!chmod 600 /root/.ssh/id_rsa
!ssh-keyscan github.com >> ~/.ssh/known_hosts
!git clone git@github.com:TimothyAlexisVass/udi.git
!rm -r sample_data

In [ ]:
%cd /content/udi
!git pull

In [ ]:
!lsof -ti:5000 | xargs kill
!python3 /content/udi/launch --share

Installing accelerate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.3 MB/s eta 0:00:00


In [ ]:
!pip install compel accelerate transformers
import os
import sys
import json
import time
import torch
import random
from udi.worker.prompt_manager import process_prompt
from udi.worker.model_manager import ModelManager
from udi.worker.filters import latents_filter
from PIL.PngImagePlugin import PngInfo

model_manager = ModelManager()
numpy_to_pil = model_manager.txt2img.numpy_to_pil

script_directory = '/content/udi/worker'
main_app_directory = os.path.dirname(script_directory)

creations_directory = os.path.join(main_app_directory, 'static/creations')
preview_directory = os.path.join(main_app_directory, 'static/preview')
active_directory = os.path.join(main_app_directory, 'worker/queue/active')
done_directory = os.path.join(main_app_directory, 'worker/queue/done')
new_directory = os.path.join(main_app_directory, 'worker/queue/new')
os.makedirs(creations_directory, exist_ok=True)
os.makedirs(preview_directory, exist_ok=True)
os.makedirs(active_directory, exist_ok=True)
os.makedirs(done_directory, exist_ok=True)
os.makedirs(new_directory, exist_ok=True)
processing_file = os.path.join(preview_directory, 'processing.json')

def console_log(message):
    print(str(message) + "\n")
def console_error(error):
    print(str(error) + "\n")
def log_process(data):
    with open(processing_file, 'w') as json_file:
        json.dump(data, json_file)

@torch.no_grad()
def decode_latents(latents):
    scaling = 4.444 + saturation / 16

    samples = model_manager.vae.decode(latents * scaling).sample
    if normalize_latents:
        samples = normalize_tensor(samples)
    else:
        samples = samples.mul(contrast/100).add(brightness/100).clamp(0, 1)
    return numpy_to_pil(samples.permute(0, 2, 3, 1).cpu().numpy())

# Globals
num_inference_steps = 20
current_step = 1
def inference_callback(index, timestep, latents):
    global current_step
    if timestep > filter_until_timestep:
        latents = latents_filter(latents, timestep, filter_type, kernel_size=filter_kernel_size, sigma=filter_spread, filter_strength=filter_strength, filter_until_timestep=filter_until_timestep)
    preview = decode_latents(latents)[0]

    if current_step < num_inference_steps:
        preview.save(os.path.join(preview_directory, f"{current_step}.jpg"), quality=70)
        log_process({'step': current_step, 'timestep': timestep.item(), 'num_inference_steps': num_inference_steps})
        current_step += 1
    else:
        current_step = 1
    previous_file = os.path.join(preview_directory, f"{current_step - 1}.jpg")
    if os.path.exists(previous_file):
        os.remove(previous_file)

    console_log(f"Step: {current_step} / {num_inference_steps} ({timestep.item()})")

    return {"latents": latents}

pipeline_defaults = {
    'num_images_per_prompt': 1,
    'output_type': 'latent',
    'callback': inference_callback,
    'callback_steps': 1,
    'guidance_rescale': 0.7,
}

width_and_height = [
    [1536, 640],
    [1344, 768],
    [1280, 832],
    [1152, 896],
    [1024, 1024],
    [896, 1152],
    [832, 1280],
    [768, 1344],
    [640, 1536],
]


In [ ]:
job_task = {'pipeline': {'prompt': 'A beautiful woman', 'negative_prompt': 'blurry, deformed, disfigured, bad anatomy, ugly', 'resolution': 4, 'guidance_scale': 8.5, 'num_inference_steps': 20, 'seed': 0}, 'inference': {'normalize_latents': False, 'upscale_result': True, 'use_refiner': True, 'saturation': 50, 'brightness': 50, 'contrast': 50, 'filter_type': 'sharpen', 'filter_strength': 5, 'filter_spread': 3, 'filter_kernel_size': 13}, 'scene': {'lighting': 'Low-Key Lighting', 'environment': '', 'ambiance': '', 'composition': '', 'elements': '', 'mood': 'Serene', 'emotion': 'Joy'}, 'view': {'shot_framing': 'Leading Lines Shot', 'focus': '', 'shot_size': 'Close-Up', 'view_angle': '', 'perspective': ''}, 'style': {'artist': '', 'medium': 'Photograph', 'technique': '', 'color_palette': 'Vibrant', 'art_movement': '', 'genre': 'Sci-fi', 'time_period': ''}, 'quality': {'quality_prompt': '', 'word_salad': {'superb': 1, 'photorealistic': 1, 'masterpiece': 1, 'intricate details even to the smallest particle': 1, 'extreme detail of the environment': 1, 'beautifully rendered textures and colors': 1, 'crisp edges': 1, 'high quality': 1}}, 'camera': {'model': 'Nikon D850', 'shutter_speed': '', 'focal_length': 'Ultra-Wide-Angle 20mm', 'aperture': '', 'sensor_format': '', 'sensitivity': '', 'film_format': '', 'film_stock': 'Kodak Portra 400'}}

global num_inference_steps

###### Define variables ###############################
pipeline = job_task["pipeline"]
scene = job_task["scene"]
view = job_task["view"]
camera = job_task["camera"]
quality = job_task["quality"]
style = job_task["style"]
file_name = pipeline["prompt"]

num_inference_steps = pipeline["num_inference_steps"]

###### Set seed and resolution parameters #############
seed = pipeline.get("seed", 0)
seed = random.randint(1, 2147483647) if seed == 0 else seed
pipeline["generator"] = torch.manual_seed(seed)
pipeline["width"], pipeline["height"] = width_and_height[pipeline.get("resolution", 4)]
if "seed" in pipeline:
    del pipeline["seed"]
if "resolution" in pipeline:
    del pipeline["resolution"]

console_log("Setting global inference parameters")
inference = {}
for inference_setting, value in job_task["inference"].items():
    globals()[inference_setting] = inference[inference_setting] = value

console_log("Building prompt...")
###### Build prompt ###################################
console_log("...scene")
if scene:
    pipeline["prompt"] += " " + " ".join(scene.values())

console_log("...view")
if view:
    view_prompt = [view.get("shot_size", ""), view.get("focus", ""), view.get("shot_framing", "")]
    if view["view_angle"]:
        view_prompt.append(f"Taken from a {view['view_angle']}")
    if view["perspective"]:
        view_prompt.append(f"with {view['perspective']}")
    pipeline["prompt"] = "(" + " ".join(view_prompt) + ")1.0 " + pipeline["prompt"]

console_log("...camera")
if camera:
    camera_prompt = []
    if camera["model"]:
        camera_prompt.append(f"taken with {camera['model']}")
    camera_prompt.append(camera.get("shutter_speed", ""))
    camera_prompt.append(camera.get("aperture", ""))
    camera_prompt.append(camera.get("focal_length", ""))
    camera_prompt.append(camera.get("sensor_format", ""))
    camera_prompt.append(camera.get("sensitivity", ""))
    if camera["film_format"]:
        camera_prompt.append(f"({camera['film_format']} film)1.0")
    camera_prompt.append(camera.get("film_format", ""))
    if camera["focal_length"]:
        camera_prompt.append(f"({camera['focal_length']} lens)1.0")
    pipeline["prompt"] = "(" + " ".join(camera_prompt) + ")1.0 " + pipeline["prompt"]

console_log("...quality")
if quality:
    if quality["word_salad"]:
        quality_string = " ".join(f"({setting}){'{0:.1f}'.format(weight)}" for setting, weight in quality["word_salad"].items())
    else:
        quality_string = ""
    pipeline["prompt"] += f" {quality['quality_prompt']} " + quality_string

console_log("...style")
if style:
    if style["genre"]:
        pipeline["prompt"] = style["genre"] + " " + pipeline["prompt"]
    if style["medium"]:
        pipeline["prompt"] = style["medium"] + " " + pipeline["prompt"]
    if style["color_palette"]:
        pipeline["prompt"] += f" {style['color_palette']} colors"
    style_prompt = [style.get("artist", ""), style.get("technique", ""), style.get("art_movement", ""), style.get("time_period", "")]
    pipeline["prompt"] += " " + " ".join(style_prompt)

###### Run Inference ##################################
prompt_1 = pipeline.pop("prompt").replace("  ", " ")
negative_prompt_1 = pipeline.pop("negative_prompt")


console_log("Running text to image model")
_ = model_manager.txt2img.to("cuda")
prompt_embeds = process_prompt(model_manager.txt2img, prompt_1=prompt_1, negative_prompt_1=negative_prompt_1)
prompt_embed_parameters = {
    "prompt_embeds": prompt_embeds["positive"]["embeds"],
    "pooled_prompt_embeds": prompt_embeds["positive"]["pooled"],
    "negative_prompt_embeds": prompt_embeds["negative"]["embeds"],
    "negative_pooled_prompt_embeds": prompt_embeds["negative"]["pooled"]
}
console_log("Setting parameters")
parameters = {**prompt_embed_parameters, **pipeline, **pipeline_defaults}
console_log("Generating latents")
latents = model_manager.txt2img(**parameters, denoising_end=0.8 if use_refiner else 1.0).images

if use_refiner:
    console_log("Switching to refiner")
    _ = model_manager.refiner.to("cuda")
    prompt_embeds = process_prompt(model_manager.refiner, prompt_1=prompt_1, negative_prompt_1=negative_prompt_1)
    prompt_embed_parameters = {
        "prompt_embeds": prompt_embeds["positive"]["embeds"],
        "pooled_prompt_embeds": prompt_embeds["positive"]["pooled"],
        "negative_prompt_embeds": prompt_embeds["negative"]["embeds"],
        "negative_pooled_prompt_embeds": prompt_embeds["negative"]["pooled"]
    }
    console_log("Setting parameters")
    parameters = {**prompt_embed_parameters, **pipeline, **pipeline_defaults}
    del parameters["width"]
    del parameters["height"]
    console_log("Generating latents")
    latents = model_manager.refiner(**parameters, denoising_start=0.8, image=latents).images

console_log("Decoding latents")
###### Decode and save image with metadata ############
image = decode_latents(latents)[0]

del pipeline["generator"]
pipeline["seed"] = seed

metadata = PngInfo()
metadata.add_text("pipeline", json.dumps(job_task["pipeline"]))
metadata.add_text("scene", json.dumps(job_task["scene"]))
metadata.add_text("view", json.dumps(job_task["view"]))
metadata.add_text("camera", json.dumps(job_task["camera"]))
metadata.add_text("quality", json.dumps(job_task["quality"]))
metadata.add_text("style", json.dumps(job_task["style"]))

console_log("Saving image")
file_name = f"{file_name[:100]}_{seed}.png"
image_path = os.path.join(creations_directory, file_name)
image.save(image_path, pnginfo=metadata)
image.save(os.path.join(preview_directory, f"{num_inference_steps}.png"))
log_process({"step": num_inference_steps, "timestep": 1, "num_inference_steps": num_inference_steps})

torch.cuda.empty_cache()

console_log("Inference completed")

In [ ]:
print(job_task)